In [1]:
import numpy as np
import random
from poker_tools import range_simulate, hand_simulate, return_hand_winner
import copy

In [2]:
##initialize
cards = set()
suits = {"Spade", "Diamond", "Heart", "Club"}
numbers = {"2":0, "3":1, "4":2, "5":3, "6":4, "7":5, "8":6, "9":7, "10":8, "J":9, "Q":10, "K":11,"A":12}

for number in numbers:
    for suit in suits:
        cards.add((suit, number))

In [3]:
set_direct = {
    "S":{("A", "K"), ("A", "Q")},
    "P":{"2", "5", "6", "A"},
    "O":{("A", "Q"), ("A", "10")}
}

hero = {("Club", "J"), ("Diamond", "Q")}
villains = {"Villain1":{}}
flop = set()
equity, win_dict = range_simulate(hero, villains, flop, cards, 100000)
print(equity)
print(win_dict)

0.57857
{'Straight Flush': 0.0, '4 of a Kind': 0.0, 'Full House': 0.04, 'Flush': 0.04, 'Straight': 0.11, '3 of a Kind': 0.07, 'Two Pair': 0.31, 'Pair': 0.37, 'High Card': 0.06}


In [4]:
hero = {("Heart", "J"), ("Heart", "10")}
villains = {"Villain":{("Club", "2"),("Diamond", "2")}}
flop = set()

equity, win_dict = hand_simulate(hero, villains, flop, cards, 100000)
print(equity)
print(win_dict)

0.54121
{'Straight Flush': 0.0, '4 of a Kind': 0.01, 'Full House': 0.1, 'Flush': 0.08, 'Straight': 0.1, '3 of a Kind': 0.12, 'Two Pair': 0.34, 'Pair': 0.24, 'High Card': 0.0}


In [2]:
from poker_simulator import PokerGame
from sample_bots import FishPlayer, RaisePlayer, CheckOrFoldPlayer
from poker_bot import PokerBotTemplate

In [71]:
class StratPlayer(PokerBotTemplate):
    def initialize_strategy(self, state):
        cards = set()
        suits = {"Spade", "Diamond", "Heart", "Club"}
        numbers = {"2":0, "3":1, "4":2, "5":3, "6":4, "7":5, "8":6, "9":7, "10":8, "J":9, "Q":10, "K":11,"A":12}

        for number in numbers:
            for suit in suits:
                cards.add((suit, number))
        
        self.cards = cards
        self.has_raised = False
    
    def play(self, state):
        flop = set() if "flop" not in state else state["flop"]
        len_players = len(state["stack_size"])
        for action in state["game_history"]:
            if action[1] == "FoldAction":
                len_players -= 1
        
        villain_dict = {}
        for i in range(len_players):
            villain_dict[i] = {}
        
        hero = state["hand"]
        
        equity, __ = range_simulate(hero, villain_dict, flop, self.cards, 10000)
        
        if self.has_raised:
            bluff = random.random() > 0.5
            if bluff:
                return ("CheckAction", 0)
            else:
                if equity < 0.5:
                    return ("FoldAction", 0)
                return ("RaiseAction", min(state["stack_size"][self.name], 5 * state["min_raise"]))

        if state["prog"] == "Preflop":
            bluff = random.random() > 0.6
            if equity < 1/(len_players * 1.5) and not bluff:
                return ("FoldAction", 0)
            else:
                self.has_raised = True
                return ("RaiseAction", 2 * state["min_raise"])
        elif state["prog"] == "Postflop":
            if equity < state["call_amount"] / (state["pot"] + state["call_amount"]):
                return ("FoldAction", 0)
            else:
                return ("CheckAction", 0)
        elif state["prog"] == "Turn":
            if equity < state["call_amount"] / (state["pot"] + state["call_amount"]):
                return ("FoldAction", 0)
            else:
                self.has_raised = True
                return ("RaiseAction", state["min_raise"])
        elif state["prog"] == "River":
            return ("CheckAction", 0)
        else:
            print ("Fail")
        
        return ("CheckAction", 0)

In [72]:
Game = PokerGame(max_players = 6)
John = StratPlayer("John")
Neil = StratPlayer("Neil")
Fold1 = CheckOrFoldPlayer("Fold1")
Game.add_player(John)
Game.add_player(Neil)
Game.add_player(Fold1)
Game.initialize_game()
for i in range(10):
    Game.play_round()
    Game.read_results(Game.recent_round)

round 1
blinds are 5,10
starting stack is 1000
----------
John has a stack of $1000 and is holding Heart10 Club2
Neil has a stack of $1000 and is holding Diamond4 Diamond2
Fold1 has a stack of $1000 and is holding Heart6 SpadeA
----------
----------
On Preflop, Fold1 did a CheckAction with $10
On Preflop, John did a FoldAction with $0
On Preflop, Neil did a FoldAction with $0
----------
winners are Fold1 with a pot of $25

round 2
blinds are 5,10
starting stack is 1000
----------
John has a stack of $995 and is holding Club10 ClubJ
Neil has a stack of $990 and is holding Club4 Spade4
Fold1 has a stack of $1015 and is holding Spade2 Spade8
----------
----------
On Preflop, John did a RaiseAction with $20
On Preflop, Neil did a RaiseAction with $80
On Preflop, Fold1 did a FoldAction with $0
On Preflop, John did a FoldAction with $0
----------
winners are Neil with a pot of $150

round 3
blinds are 5,10
starting stack is 1000
----------
John has a stack of $965 and is holding Spade9 ClubA

In [6]:
Game = PokerGame(max_players = 6)
Raise1 = RaisePlayer("Raise1")
CheckFold1 = CheckOrFoldPlayer("Fold1")
Raise2 = RaisePlayer("Raise2")
Fish1 = FishPlayer("Derrick")
Fish2 = FishPlayer("John")
Fish3 = FishPlayer("Jeff")
Game.add_player(CheckFold1)
Game.add_player(Fish1)
Game.add_player(Fish2)
Game.add_player(Fish3)
Game.add_player(Raise1)
Game.add_player(Raise2)
Game.initialize_game()
for i in range(1000):
    Game.play_round()